In [ ]:
import json
import pandas as pd
import matplotlib as plt
from pycoingecko import CoinGeckoAPI

cg = CoinGeckoAPI()
pd.options.display.float_format = '{:.2f}'.format 

df = pd.read_json(r'dataset\response_1692132599095.json', dtype='unicode')

In [ ]:
#testeo si funciona la conexion con la api
cg.ping()

In [ ]:
crypto_lista = cg.get_coins_list()
df_cryptos= pd.DataFrame.from_dict(crypto_lista)

df_cryptos

In [ ]:
cryptos_categorias = pd.DataFrame(cg.get_coins_categories_list())
cryptos_categorias

In [ ]:

#considero que no voy a utilizar estas columnas:
df.drop(columns=['image','last_updated', 'roi', 'ath_date','atl_date',  'high_24h',
                 'low_24h','price_change_24h','price_change_percentage_24h','market_cap_change_24h',
                 'market_cap_change_percentage_24h'], inplace=True)

In [ ]:
coins_lista=df["id"].tolist()



In [34]:
#con esta funcion obtengo una lista con el precio de una crypto por dia en el lapso de un año
#la idea seria ver que tanto varia el precio a lo largo de un año y de eso armar un numero que signifique la volatilidad
#de esa crypto moneda como una variable importante para determinar que cryptomonedas son mas estables que otras

def get_price_per_day(coin_id):
    price_per_day=[]
    row=cg.get_coin_market_chart_by_id(id= coin_id, vs_currency = 'usd', days ='365', interval='daily', precision='5')
    for i in range(len(row['prices'])):
        price_per_day.append(ejemplo['prices'][i][1])
        
    return price_per_day

In [35]:
coins_performance=get_price_per_day('bitcoin')
coins_performance

[21175.23,
 21615.76,
 21387.75,
 21562.45,
 21395.46,
 21618.01,
 20271.32,
 20069.95,
 19659.25,
 20308.86,
 19805.35,
 20023.55,
 20153.98,
 19941.45,
 19814.56,
 19980.01,
 19785.73,
 18859.5,
 19280.63,
 19322.78,
 21359.58,
 21707.39,
 21739.8,
 22339.67,
 20184.97,
 20255.92,
 19702.17,
 19764.41,
 20131.68,
 19437.16,
 19570.39,
 18869.93,
 18539.64,
 19464.32,
 19291.9,
 18939.97,
 18809.41,
 19219.73,
 19116.33,
 19444.79,
 19563.77,
 19476.93,
 19314.46,
 19065.15,
 19620.57,
 20345.4,
 20161.83,
 19949.05,
 19457.31,
 19418.04,
 19448.15,
 19142.69,
 19058.63,
 19153.04,
 19383.94,
 19198.11,
 19072.78,
 19272.92,
 19557.9,
 19348.03,
 19134.24,
 19031.16,
 19173.28,
 19203.87,
 19574.82,
 19316.94,
 20095.15,
 20774.25,
 20278.37,
 20591.24,
 20800.71,
 20623.87,
 20494.6,
 20489.65,
 20162.98,
 20210.91,
 21150.32,
 21282.99,
 20907.0,
 20597.76,
 18562.35,
 15742.44,
 17594.92,
 17080.22,
 16797.99,
 16344.34,
 16645.83,
 16890.53,
 16689.4,
 16718.36,
 16707.02,
 16712.

# Clasificación de las variables

### cuantitativos
* discretos = enteros
* continuos = se divide infinitamente

### cualitativos
* categoricos = de una categoria o clase especifica
* ordinales = de categoria en la que IMPORTA un orden o jerarquía

### Categoricas 

* id
* symbol
* name

### Discretas
* Ninguna

### Continuas  

* current_price 
* market_cap 
* fully_diluted_valuation --> si todas las monedas de una crypto estuvieran en circulacion, es para saber el potencial de una moneda con su valor actual
* total_volume 
* high_24h 
* low_24h
* circulating_supply
* total_supply
* max_supply
* ath
* ath_change_percentage
* atl
* atl_change_percentage
* ath_date
* atl_date
* roi
* price_change_24h
* price_change_percentage_24h
* market_cap_change_24h
* market_cap_change_percentage_24h

### Ordinales 

* market_cap_rank

## Conclusión

No hay variables discretas en mi opinion ya que si bien variables como circulating supply usualmente se refiere a una cantidad espcífica de monedas en circulación, cualquier moneda puede ser separada en partes más pequeñas de forma infinita, hence, es una variable continua.

La gran mayoría de variables son continuas ya que se trata de dinero o cryptos que, como ya dije, pueden dividirse infinitamente.

Cabe recalcar la importancia de la variable ordinal "market_cap_rank" ya que juega un papel muy importante en el mundo de las cryptos aquellas rankeadas en lo alto. Esto se debe a que ganan mayor fama y reconocimiento. Esta fama es incluso mayor si se trata de una crypto que logra mantenerse durante largos periodos de tiempo (años) en dicho ranking. Por ejemplo bitcoin, ethereum, usdt, etc.




In [ ]:

df.columns.tolist()

In [ ]:
#considero que para poder continuar, debo poner el tipo de dato que corresponde a cada columna

strings= ["id", "symbol", "name"]
floats=['current_price',
 'market_cap',
 'fully_diluted_valuation',
 'total_volume',
 'circulating_supply',
 'total_supply',
 'max_supply',
 'ath',
 'ath_change_percentage',
 'atl',
 'atl_change_percentage']

for i in strings:
    df[i].fillna(0)
    df[i] = df[i].astype("string")

for i in floats:
    df[i].replace('None', pd.NA, inplace=True)
    df[i].fillna(0, inplace=True)
    df[i] = df[i].astype("float")

df["market_cap_rank"] = df["market_cap_rank"].astype("int")

df.dtypes

# Como quiero plantear este EDA

Mi objetivo en este EDA es obtener insights sobre las 10 mejores criptomonedas como potencial inversion para clientes de una empresa.

Este analisis exploratorio de los datos va a estar dividido en dos partes:

* Análisis de Bitcoin y sus altcoins (altcoins = aquellas criptomonedas que no son Bitcoin)

* Análisis de stable coins (aquellas criptomonedas que tienen su valor anclado al dólar)

Este acercamiento se debe a que por un lado el comportamiento de las altcoins es muy diferente al de las stablecoins. Lo que quiero lograr es marcar una diferencia entre aquellas criptos que son más riesgosas pero mucho más volátiles y aquellas que son estables pero que no van a suponer un retorno alto.

In [ ]:
#divido mi dataframe en stablecoins y criptomonedas

df_stablecoins = df[(df["current_price"] > 0.99) & (df["current_price"] < 1)]

df_altcoins_and_btc = df[~((df["current_price"] >= 0.99) & (df["current_price"] <= 1))] #usar ~ para negar una condicion 
                                                                                #se hace con Alt Gr + (boton de asterisco al lado del enter)


df_altcoins_and_btc


In [ ]:
df.describe()
#busco informacón relevante que me ayude a comenzar a filtrar ciertas criptomonedas

In [ ]:
#df.drop(df[df['name'] == 'Bitcoin'].index, inplace=True)
#df.drop(df[df['name'] == 'Ethereum'].index, inplace=True)


#Plot histograms of numerical data
%matplotlib inline
import matplotlib.pyplot as plt
df.hist(bins = 50, figsize = (20,15))
plt.show